# Cross-validation

In [69]:
from datetime import timedelta
import numpy as np
import polars as pl
import pandas as pd


from tools import load_data_actions, generate_lightfm_recs_mapper, recall_at
from tqdm import tqdm

from rectools import Columns

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

import mlflow
from mlflow.models import infer_signature 

from catboost import Pool, CatBoostRanker

SEED = 42
top_N = 40
DATA_DIR = 'data/'

In [3]:
def dataframe2rectools(df):
    return (
        df[["cookie", "node", "event_date"]]
        .with_columns(pl.lit(1).alias('weight'))
        .rename({
            "cookie": Columns.User,
            "node": Columns.Item,
            "weight": Columns.Weight,
            "event_date": Columns.Datetime,
        })
    ).to_pandas()

def cast_int_dtypes(df):
    columns = ["cookie", "node"]
    for col in columns:
        df = df.with_columns(pl.col(col).cast(pl.Int64))
    return df

In [4]:
# Base
# ------ downloading
df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')
df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features_preproc_20.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

# train-validation split
df_train, df_eval = load_data_actions(df_clickstream, df_event)

# train-validation postprocessing
df_train = cast_int_dtypes(df_train)
df_eval = cast_int_dtypes(df_eval)

In [7]:
# item features
# ------ mean text features
mean_by_cat = (
    df_text_features
    .join(df_cat_features["item", "node"], on="item", how="left")
    .to_pandas()
    .groupby("node")["title_projection"]
    .apply(lambda vs: np.mean(np.stack(vs.values), axis=0).tolist())
    .reset_index(name="mean_title_projection")
)
# ------ additional features
addit_features = pl.read_parquet(DATA_DIR+"cat_features_preproc_20.pq")
atr_cols = [i for i in addit_features.columns if "attr" in i]
addit_features = (
    addit_features.group_by("node")
    .agg([pl.col(atr).mean() for atr in atr_cols])
)
# ------ concatination
item_features = pl.DataFrame(mean_by_cat)
item_features = item_features.with_columns(pl.col("mean_title_projection").list.to_struct()).unnest("mean_title_projection")
item_features = item_features.join(addit_features, how="left", on="node")

/var/folders/1v/2njvtrl903s020_gk0tw647c0000gn/T/ipykernel_1215/2199896430.py:20: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  item_features = item_features.with_columns(pl.col("mean_title_projection").list.to_struct()).unnest("mean_title_projection")


In [8]:
# Candidates from 1st steps
candidates_ALS = pl.read_csv('data/first_stage_candidates_ALS_200.csv')
candidates_ANN = pl.read_csv('data/first_stage_prediction_ANN_40.csv')
candidates_BM25 = pl.read_csv('data/first_stage_prediction_BM25_40.csv')

candidates = pl.concat([candidates_ALS, candidates_BM25, candidates_ANN])
candidates = candidates.unique(subset=["cookie", "node"], maintain_order=True)
candidates = candidates.with_columns(
    pl.col("cookie").cum_count().over("cookie").alias("rank").cast(pl.Int64)
)

In [41]:
# Consructing dataframes for learning
select_col = ["cookie", "node", "rank", 'target']
# ------ positive interactions
pos = (
    candidates
    .join(
        df_eval,
        on=['cookie', 'node'],
        how='inner'
    )
    .with_columns(
        pl.lit(1).alias("target")
    )
    .select(select_col)
)

# ------ negative interactions
neg = (
    candidates
    .join(
        df_eval,
        on=['cookie', 'node'],
        how='left'
    )
    .filter(pl.col("event").is_null())
    .with_columns(
        pl.lit(0).alias("target")
    )
    .sample(fraction=0.03)
    .select(select_col)
)

In [102]:
kfold = KFold(n_splits=5)
drop_col = ['cookie', 'node']
target_col = ['target']
cat_col = [f"field_{i}" for i in range(64)]

parameters = {
    'subsample': 0.9,
    'max_depth': 4,
    'n_estimators': 301,
    'learning_rate': 0.03,
    'thread_count': 20,
    'random_state': SEED,
    'verbose': 300,
    'loss_function': 'YetiRank',  
    "eval_metric":'AUC'
}

for fold_num, (train_u, valid_u) in enumerate(kfold.split(df_eval.select('cookie').unique())):
    X_train = pl.concat([
        pos.filter(pl.col("cookie").is_in(train_u)), neg.filter(pl.col("cookie").is_in(train_u))
        ]).sample(fraction=1, shuffle=True).sort(by="cookie")
    X_val = pl.concat([
        pos.filter(pl.col("cookie").is_in(valid_u)), neg.filter(pl.col("cookie").is_in(valid_u))
        ]).sample(fraction=1, shuffle=True).sort(by="cookie")
    
    print(sum(X_train["cookie"].unique().is_in(X_val["cookie"].unique())))
    train_cookies = X_train.select("cookie")
    X_train, y_train = X_train.drop(drop_col + target_col), X_train.select(target_col)

    val_cookies = X_val.select("cookie")
    y_val = X_val.select(target_col)

    train_pool = Pool(
        data=X_train.to_pandas(),
        label=y_train.to_pandas(),
        group_id=train_cookies.to_pandas(),
        cat_features=["rank"]
    )

    model = CatBoostRanker(**parameters)
    model.fit(train_pool)
    pred_scores = model.predict(X_val[X_train.columns].to_pandas())
    prediction = X_val.with_columns(pl.Series("score", pred_scores)).sort(by=["cookie", "score"], descending=[True, False])
    boost_recall = recall_at(df_eval.filter(pl.col("cookie").is_in(valid_u)), prediction, k=40)
    print(boost_recall)
    pass

0
Groupwise loss function. OneHotMaxSize set to 10
0:	total: 24.2ms	remaining: 7.26s


KeyboardInterrupt: 

In [99]:
df_eval['cookie', 'node'].unique().join(prediction, on='cookie', how='semi')

cookie,node
i64,i64
7427,87062
9869,2103
5632,106609
9312,214324
10537,120269
…,…
3601,280663
1916,120267
10633,214510


In [95]:
df_eval['cookie', 'node'].unique()

cookie,node
i64,i64
54891,214235
106413,214232
125508,200252
89409,153140
68029,194770
…,…
68388,60158
145483,85872
21282,198912


In [96]:
prediction

cookie,node,rank,target,score
i64,i64,i64,i32,f64
0,264373,223,0,-0.383472
0,130802,2,0,1.946676
0,117531,55,0,0.363641
0,259469,208,0,-0.654767
0,115715,14,0,1.058767
…,…,…,…,…
11012,136804,171,0,-0.383472
11012,259473,221,0,-0.654767
11012,214231,119,0,0.07932


In [94]:
recall_at(df_eval['cookie', 'node'].unique().join(prediction, on='cookie', how='semi'), prediction, k=40)

0.3295400209841646

In [ ]:
X_val

cookie,node,rank,target
i64,i64,i64,i32
10388,130608,205,0
656,155883,221,0
4048,153184,222,0
8287,221240,13,0
3301,214253,149,0
…,…,…,…
9771,28803,121,0
1584,116139,156,0
1120,1900,225,1


In [84]:
train_cookies.to_pandas()

,cookie
0,16107
1,13254
2,49013
3,11795
4,42017
...,...
135489,47471
135490,44330
135491,53430
135492,15633


In [ ]:
# positive interactions
pos = candidates.merge(
    df_eval,
    on=['cookie', 'node'],
    how='inner'
    )

pos.loc[:, 'target'] = 1
print(pos.shape)
# negative interactions
neg = candidates.set_index(['cookie', 'node'])\
        .join(df_eval.set_index(['cookie', 'node']))

neg = neg[neg['event'].isnull()].reset_index()
neg = neg.sample(frac=0.03)
neg.loc[:, 'target'] = 0

print(neg.shape)


# train test
ctb_train_users, ctb_test_users = train_test_split(
    df_eval['cookie'].unique(),
    random_state=SEED,
    test_size=0.2
    )
# train eval
# ctb_train_users, ctb_eval_users = train_test_split(
#     ctb_train_users,
#     random_state=SEED,
#     test_size=0.1
#     )

select_col = ["cookie", "node", "rank", 'target']


# Catboost train
ctb_train = shuffle(
    pd.concat([
        pos[pos['cookie'].isin(ctb_train_users)],
        neg[neg['cookie'].isin(ctb_train_users)]
])[select_col]
)
# Catboost test
ctb_test = shuffle(
    pd.concat([
        pos[pos['cookie'].isin(ctb_test_users)],
        neg[neg['cookie'].isin(ctb_test_users)]
])[select_col]
)
# # for early stopping
# ctb_eval = shuffle(
#     pd.concat([
#         pos[pos['cookie'].isin(ctb_eval_users)],
#         neg[neg['cookie'].isin(ctb_eval_users)]
# ])[select_col]
# )

(47261, 5)
(413161, 5)


In [ ]:
ctb_train['target'].value_counts(normalize=True), ctb_test['target'].value_counts(normalize=True)

(target
 0    0.897621
 1    0.102379
 Name: proportion, dtype: float64,
 target
 0    0.896279
 1    0.103721
 Name: proportion, dtype: float64)

In [ ]:
item_features = item_features.to_pandas()
user_col = ['cookie']
item_col = item_features.columns

train_feat = (
    ctb_train
    # .merge(
    #     users[user_col],
    #     on=['user_id'],
    #     how='left')
    .merge(
        item_features[item_col],
        on=['node'],
        how='left')
)
# eval_feat = (
#     ctb_eval
#     # .merge(
#     #     users[user_col],
#     #     on=['user_id'],
#     #     how='left')
#     .merge(
#         item_features[item_col],
#         on=['node'],
#         how='left')
# )
test_feat = (
    ctb_test
    # .merge(
    #     users[user_col],
    #     on=['user_id'],
    #     how='left')
    .merge(
        item_features[item_col],
        on=['node'],
        how='left'
        )
)
train_feat.isna().sum().sum(), test_feat.isna().sum().sum()

AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [ ]:
train_feat = train_feat.sort_values(by=["cookie", "target", "rank"], ascending=[True, False, True])
# eval_feat = eval_feat.sort_values(by=["cookie", "target", "rank"], ascending=[True, False, True])
test_feat = test_feat.sort_values(by=["cookie", "target", "rank"], ascending=[True, False, True])

In [ ]:
drop_col = ['cookie', 'node']
target_col = ['target']
cat_col = [f"field_{i}" for i in range(64)]

X_train, y_train = train_feat.drop(drop_col + target_col, axis=1), train_feat[target_col]
train_cookies = train_feat["cookie"]
# X_val, y_val = eval_feat.drop(drop_col + target_col, axis=1), eval_feat[target_col]
# val_cookies = eval_feat["cookie"]
X_test, y_test = test_feat.drop(drop_col + target_col, axis=1), test_feat['target']
test_cookies = test_feat["cookie"]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((368512, 85), (368512, 1), (91910, 85), (91910,))

In [ ]:
from catboost import Pool
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=train_cookies,
    cat_features=["rank"]
)
# val_pool = Pool(
#     data=X_val,
#     label=y_val,
#     group_id=val_cookies
# )
test_pool = Pool(
    data=X_test,
    label=y_test,
    group_id=test_cookies,
    cat_features=["rank"]
)

In [ ]:
from catboost import CatBoostClassifier, CatBoostRanker
from copy import deepcopy

# параметры для обучения
est_params = {
  'subsample': 0.9,
  'max_depth': 4,
  'n_estimators': 5000,
  'learning_rate': 0.03,
  'thread_count': 20,
  'random_state': SEED,
  'verbose': 200,
#   "loss_function":'YetiRankPairwise',
#  "eval_metric":'NDCG'
}

ctb_model = CatBoostClassifier(**est_params)


default_parameters = {
    'iterations': 5000,
    'custom_metric': ['NDCG', 'AUC'],
    'verbose': 100,
    # "loss_function":'YetiRankPairwise',
    'thread_count': 20,
    'learning_rate': 0.1,
    'max_depth': 4,
    'random_seed': SEED,
}

parameters = {}

def fit_model(loss_function, additional_params=None, train_pool=None, test_pool=None):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function

    if additional_params is not None:
        parameters.update(additional_params)

    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True, early_stopping_rounds=100,)

    return model

In [ ]:
model = fit_model(
    'RMSE',
    {'custom_metric': ['PrecisionAt:top=40', 'RecallAt:top=40', 'MAP:top=40', 'AUC']},
    train_pool=train_pool,
    test_pool=test_pool
    )

/opt/anaconda3/envs/recsys/lib/python3.11/site-packages/catboost/core.py:6402: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.3014027	test: 0.3031835	best: 0.3031835 (0)	total: 26ms	remaining: 2m 9s
100:	learn: 0.2891574	test: 0.2908572	best: 0.2908572 (100)	total: 2.62s	remaining: 2m 7s
200:	learn: 0.2878517	test: 0.2898062	best: 0.2898062 (200)	total: 5.09s	remaining: 2m 1s
300:	learn: 0.2870966	test: 0.2893645	best: 0.2893645 (300)	total: 7.66s	remaining: 1m 59s
400:	learn: 0.2865652	test: 0.2891557	best: 0.2891557 (400)	total: 10.2s	remaining: 1m 57s
500:	learn: 0.2861316	test: 0.2890014	best: 0.2890014 (500)	total: 12.7s	remaining: 1m 54s
600:	learn: 0.2857741	test: 0.2889353	best: 0.2889353 (600)	total: 15.3s	remaining: 1m 51s
700:	learn: 0.2854879	test: 0.2888529	best: 0.2888529 (700)	total: 17.8s	remaining: 1m 49s
800:	learn: 0.2852241	test: 0.2887887	best: 0.2887887 (800)	total: 20.5s	remaining: 1m 47s
900:	learn: 0.2849846	test: 0.2887726	best: 0.2887686 (846)	total: 23s	remaining: 1m 44s
1000:	learn: 0.2847254	test: 0.2887414	best: 0.2887405 (950)	total: 25.5s	remaining: 1m 41s
1100:	le

In [ ]:
from catboost import CatBoostClassifier, CatBoostRanker
from catboost import Pool
# параметры для обучения
est_params = {
    'subsample': 0.9,
    'max_depth': 4,
    'n_estimators': 5000,
    'learning_rate': 0.03,
    'thread_count': 20,
    'random_state': SEED,
    'verbose': 200,
    "loss_function": 'YetiRank',
}
def fit_model(train_pool, df_eval, parameters=None):
    model = CatBoostRanker(**parameters)
    model.fit(train_pool)
    return model



In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("avito_cup_pers_recs")
with mlflow.start_run():
    roc_auc = roc_auc_score(y_test, y_pred)

    mlflow.log_params(model.get_all_params())
    mlflow.log_metric("roc_auc", roc_auc)

    mlflow.set_tag("Training Info", "Final CatboostRanker model of competition")

    model_info = mlflow.catboost.log_model(
        model,
        "ctb_model_ranker",
        signature=signature,
        registered_model_name="comp-final-ranker",
        )

    print(f"ROC AUC score = {roc_auc:.4f}")